In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import squarify

In [4]:
#데이터 불러오기
data = pd.read_csv('/Users/namchangseok/Desktop/코드스테이츠/AI_12기/cp2/2019-Oct.csv',usecols=['event_time','event_type','price','user_id','user_session'])
data

,event_time,event_type,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...
42448759,2019-10-31 23:59:58 UTC,view,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
42448760,2019-10-31 23:59:58 UTC,view,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8
42448761,2019-10-31 23:59:58 UTC,view,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd
42448762,2019-10-31 23:59:59 UTC,view,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204


In [8]:
data['event_type'].unique()
data=data.loc[data.event_type == 'purchase']

In [5]:
data['event_time']=pd.to_datetime(data['event_time']).dt.tz_convert(None)

In [17]:
import pyarrow.feather as feather
os.makedirs('tmp', exist_ok=True)  # Make a temp dir for storing the feather file
feather.write_feather(data, './tmp/data')

In [21]:
%%time
#load the feather data cause feather more lightweight
data = pd.read_feather('./tmp/data')
data

CPU times: user 210 ms, sys: 165 ms, total: 375 ms
Wall time: 619 ms


,event_time,event_type,price,user_id,user_session
162,2019-10-01 00:02:14,purchase,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564
308,2019-10-01 00:04:37,purchase,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68
379,2019-10-01 00:06:02,purchase,29.51,514591159,0e5dfc4b-2a55-43e6-8c05-97e1f07fbb56
442,2019-10-01 00:07:07,purchase,54.42,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f
574,2019-10-01 00:09:26,purchase,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553
...,...,...,...,...,...
42448148,2019-10-31 23:55:13,purchase,102.45,516604399,a98bd433-57ac-42d0-ba81-e18b135a7a16
42448173,2019-10-31 23:55:21,purchase,20.59,514622109,5724116e-365b-4ac1-9d03-b8d66e1ccc7c
42448271,2019-10-31 23:56:03,purchase,577.89,512717356,f35ac37c-9573-4e30-b3d9-c09bb0b95a2b
42448362,2019-10-31 23:56:53,purchase,1054.60,533892594,3a5a3b01-2ab1-4a1d-a202-30d336e0057b


In [8]:
'''
current_day= pd.to_datetime('20191001')
time_diff = data['event_time']-current_day
seconds= [x.total_seconds() for x in time_diff]
data['event_time']=seconds
'''

In [24]:
data['event_time'].max()

2678356.0

In [22]:
data=data.groupby(by='user_session').agg(Date_order=('event_time',lambda x: x.max()),
                                        user_id=('user_id',lambda x: x.unique()),
                                        Quantity=('user_session','count'),
                                        money_spent=('price','sum'))

In [1]:
data

NameError: name 'data' is not defined

In [23]:
import datetime as dt
study_date = dt.datetime(2019,12,1)
data=pd.DataFrame(data)
data['last_purchase']=study_date - data['Date_order']
data['last_purchase'].astype('timedelta64[D]')
data['last_purchase']=data['last_purchase'] / np.timedelta64(1, 'D')
data.head()

,Date_order,user_id,Quantity,money_spent,last_purchase
user_session,,,,,
00000083-8816-4d58-a9b8-f52f54186edc,2019-10-06 11:34:30,546521725,1,289.52,55.517708
000001fd-1f89-45e8-a3ce-fe3218cabfad,2019-10-25 08:39:11,560486342,1,171.90,36.639456
00004ada-8f93-49a6-956d-4ed71ae94791,2019-10-17 13:37:59,560744406,2,379.78,44.431956
00005026-a9d1-4e2b-8290-3cc14e4bad89,2019-10-15 08:09:02,520649833,1,90.07,46.660394
00005b76-13ba-4afe-b80d-2f2b337d3e92,2019-10-23 14:27:08,534210306,2,571.92,38.397824


In [24]:
RFM= data.groupby('user_id').agg(Recency=('last_purchase',lambda x: x.min()),
                                 Frequency=('user_id',lambda x: x.count()),
                                 Monetary=('money_spent',lambda x: x.sum()))
RFM.head()

,Recency,Frequency,Monetary
user_id,,,
264649825,55.141539,2,1240.04
303160429,48.440637,1,340.59
340041246,47.384965,3,915.52
371877401,52.339491,1,29.89
384989212,43.527789,1,41.44


In [26]:
del data

In [ ]:
# 유